In [14]:
import os
import pandas as pd
import os
import nltk
from nltk.tokenize import sent_tokenize
import joblib
from transformers import RobertaTokenizer, RobertaForSequenceClassification, pipeline
import torch

nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wantu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
label_encoder = joblib.load(r'C:\Users\wantu\Desktop\hse\роберта без менандра1\label_encoder.pkl')

C:\Users\wantu\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [11]:
tokenizer = RobertaTokenizer.from_pretrained('wantuta/roberta_classifier_menander2')
model = RobertaForSequenceClassification.from_pretrained('wantuta/roberta_classifier_menander2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
def classify_text(text, tokenizer, model, label_encoder):
    sentences = sent_tokenize(text)
    classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)
    results = classifier(sentences, truncation=True, max_length=512)
    for result in results:
        result['label'] = label_encoder.inverse_transform([int(result['label'].split('_')[-1])])[0]
    return results

In [15]:
directory_path = r'C:\Users\wantu\Desktop\hse\menan132 главы'

chapter_authors = {}

for filename in os.listdir(directory_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            classified_data = classify_text(text, tokenizer, model, label_encoder)
            chapter_authors[filename] = classified_data

for chapter, data in chapter_authors.items():
    author_counts = {}
    total_sentences = len(data)
    
    for item in data:
        label = item['label']
        author_counts[label] = author_counts.get(label, 0) + 1
        
    sorted_authors = sorted(author_counts.items(), key=lambda x: x[1], reverse=True)
    df = pd.DataFrame(sorted_authors, columns=['Author', 'Count'])
    df['Percentage'] = (df['Count'] / total_sentences * 100).round(2)
    df['Chapter'] = chapter
    
    print(f"Results for {chapter}:")
    print(df)
    print("\n")

Results for 1.txt:
       Author  Count  Percentage Chapter
0      Others     11       68.75   1.txt
1    Menander      3       18.75   1.txt
2  Hermogenes      1        6.25   1.txt
3   Demetrius      1        6.25   1.txt


Results for 10.txt:
                       Author  Count  Percentage Chapter
0                      Others     11       52.38  10.txt
1                    Menander      7       33.33  10.txt
2                   Demetrius      2        9.52  10.txt
3  Dionysius Halicarnassensis      1        4.76  10.txt


Results for 11.txt:
                       Author  Count  Percentage Chapter
0                      Others     28       49.12  11.txt
1                    Menander     12       21.05  11.txt
2                   Demetrius      9       15.79  11.txt
3            Aelius Aristides      3        5.26  11.txt
4  Dionysius Halicarnassensis      2        3.51  11.txt
5                  Hermogenes      2        3.51  11.txt
6            Valerius Apsines      1        1.75